In [15]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math

from DataLoader import *
from Node import *

device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [16]:
file_name = 'dataset/ds10k.csv' # dataset/ds1m.csv

# Hyperparameters
epochs = 1                  # 1
batch_size = 128            # 128
learning_rate = 1e-4        # 1e-4

dropout = 0.1               # 0.1
embedding_dim = 128         # 128
ff_dim = 2048               # 2024
num_heads = 8               # 8
encoder_layers = 6          # 6
decoder_layers = 6          # 6

In [17]:
data = DataLoader(pd.read_csv(file_name))
train, validate = data.split(0.9)

In [18]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:,:x.size(1)]
        return x
    
class Model(nn.Module):
    
    def __init__(self, vocab_size, encoder_size=5000, decoder_size=5000, d_model=128, ff_dim= 2048, num_heads= 8, encoder_layers= 6, decoder_layers= 6, dropout= 0.1):
        super(Model, self).__init__()
    
        self._embedding_encoder = nn.Embedding(vocab_size, d_model)
        self._embedding_decoder = nn.Embedding(vocab_size, d_model)

        self._positional_encoder = PositionalEncoding(d_model, encoder_size)
        self._positional_decoder = PositionalEncoding(d_model, decoder_size)

        self._transformer = nn.Transformer(d_model,num_heads, encoder_layers, decoder_layers, ff_dim, batch_first=True, dropout= dropout)
        self._fc = nn.Linear(d_model, vocab_size)

    def forward(self, x, y, mask = None, encoder_mask=None, decoder_mask=None):
        x = self._positional_encoder(self._embedding_encoder(x))
        y = self._positional_decoder(self._embedding_decoder(y))

        output = self._transformer(x, y, tgt_mask=mask,src_key_padding_mask=encoder_mask,tgt_key_padding_mask=decoder_mask)
        
        output = self._fc(output)
        return F.softmax(output, dim=-1)

In [19]:
def train_model(model, mask, optimizer, PAD_ID, ds):
    losses = []
    print('Train:')
    for encoder_input, decoder_input, decoder_output, _, _ in DataLoader.batch(ds, batch_size):
        print(f'Total: {len(ds['encoder_input'])}, Current: {batch_size * len(losses)}, Loss: {np.mean(losses[-10:]) if len(losses) >= 10 else 0} ', end='\r')
            
        encoder_input = torch.tensor(encoder_input).to(device)
        decoder_input = torch.tensor(decoder_input).to(device)
        decoder_output = torch.tensor(decoder_output).to(device)

        output = model(encoder_input, decoder_input, mask)
        loss = F.cross_entropy(output.view(-1, output.size(-1)), decoder_output.view(-1), ignore_index=PAD_ID)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    print()

In [20]:
def validate_model(model, mask, PAD_ID, ds):
    with torch.no_grad():
        losses = []
        print('Validate:')
        for encoder_input, decoder_input, decoder_output, _, _ in DataLoader.batch(ds, batch_size):
            print(f'Total: {len(ds['encoder_input'])}, Current: {batch_size * len(losses)}', end='\r')
                
            encoder_input = torch.tensor(encoder_input).to(device)
            decoder_input = torch.tensor(decoder_input).to(device)
            decoder_output = torch.tensor(decoder_output).to(device)

            output = model(encoder_input, decoder_input, mask)
            loss = F.cross_entropy(output.view(-1, output.size(-1)), decoder_output.view(-1), ignore_index=PAD_ID)

            losses.append(loss.item())
    print()
    return np.mean(losses)

In [21]:
def test_model(model, ds, n):
    valid, correct = 0, 0
    if len(ds) < n: n = len(ds)

    with torch.no_grad():
        for i in range(n):
            seq = [2]
            encoder_input = torch.tensor([ds['encoder_input'][i]]).to(device)
            encoder_target = data._decode(ds['decoder_output'][i][:ds['decoder_output'][i].index(3)])

            for _ in range(data._decoder_size):
                decoder_input = torch.tensor([seq]).to(device)
                output = torch.multinomial(model(encoder_input, decoder_input)[-1][- 1], num_samples=1).item()
                if output == 3:
                    break
                else:
                    seq +=[output]

            target_node = Node.parse(encoder_target)
            output_node = Node.parse(data._decode(seq)[1:])
            
            if output_node:
                valid += 1
                if output_node.key == target_node.key:
                    correct += 1
    return valid/n, correct/n

In [22]:
model = Model(
    vocab_size=len(data),
    d_model=embedding_dim,
    ff_dim=ff_dim,
    num_heads=num_heads,
    encoder_layers=encoder_layers,
    decoder_layers=decoder_layers,
    dropout=dropout).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
mask = nn.Transformer.generate_square_subsequent_mask(data._decoder_size, device=device)
PAD_ID = torch.tensor(0).to(device)

print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')    

7.520784 M parameters


In [23]:
for epoch in range(epochs):
    train_model(model, mask, optimizer, PAD_ID, train)
    print(validate_model(model, mask, PAD_ID, validate))

Train:
Total: 9000, Current: 8960, Loss: 2.3418898582458496 
Validate:
Total: 1000, Current: 896
2.3351920545101166


In [24]:
test_score = test_model(model, validate, 100)
print(f'Valid outputs: {test_score[0]}, Correct outputs: {test_score[1]}')

Valid outputs: 0.6, Correct outputs: 0.0
